# OSOAA Radiative Transfer Simulation Tutorial
## Focus: Water-Leaving Radiance

This notebook guides you through running OSOAA (Ocean Successive Orders with Atmosphere - Advanced) simulations and analyzing water-leaving radiance outputs.

**OSOAA** is a vector radiative transfer model for coupled atmosphere-ocean systems. It computes radiance and polarization using the successive orders of scattering method.

### What you'll learn:
1. Setting up OSOAA simulation parameters
2. Running the Fortran executable from Python
3. Parsing and visualizing water-leaving radiance
4. Understanding the effect of chlorophyll concentration
5. Exploring spectral and angular dependencies

## 1. Setup and Configuration

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import os
import tempfile
from pathlib import Path
from typing import Dict, Optional, List

# Set up paths - adjust OSOAA_ROOT to your installation
OSOAA_ROOT = Path(os.environ.get('OSOAA_ROOT', Path.cwd().parent))
OSOAA_EXE = OSOAA_ROOT / 'exe' / 'OSOAA_MAIN.exe'
OSOAA_FIC = OSOAA_ROOT / 'fic'

print(f"OSOAA Root: {OSOAA_ROOT}")
print(f"Executable exists: {OSOAA_EXE.exists()}")
print(f"Data files directory exists: {OSOAA_FIC.exists()}")

## 2. OSOAA Python Wrapper Class

We'll create a simple Python class to interface with the OSOAA Fortran executable.

In [ ]:
class OSOAASimulation:
    """
    Python wrapper for OSOAA radiative transfer simulations.
    
    This class handles:
    - Parameter file generation
    - Execution of the Fortran code
    - Parsing of output files
    """
    
    def __init__(self, osoaa_root: Path, work_dir: Optional[Path] = None):
        self.osoaa_root = Path(osoaa_root)
        self.exe_path = self.osoaa_root / 'exe' / 'OSOAA_MAIN.exe'
        self.fic_path = self.osoaa_root / 'fic'
        self.work_dir = Path(work_dir) if work_dir else Path(tempfile.mkdtemp())
        
        if not self.exe_path.exists():
            raise FileNotFoundError(f"OSOAA executable not found at {self.exe_path}")
    
    def get_default_params(self, wavelength: float = 550.0, 
                           solar_zenith: float = 30.0,
                           chlorophyll: float = 0.1,
                           aot: float = 0.1,
                           wind_speed: float = 5.0) -> Dict:
        """
        Generate default simulation parameters.
        
        Parameters
        ----------
        wavelength : float
            Simulation wavelength in nm (299-1000 nm)
        solar_zenith : float
            Solar zenith angle in degrees (0-90)
        chlorophyll : float
            Chlorophyll-a concentration in mg/m³
        aot : float
            Aerosol optical thickness at reference wavelength
        wind_speed : float
            Wind speed in m/s for sea surface roughness
        """
        return {
            # Wavelength
            'OSOAA.Wa': wavelength,
            
            # Viewing geometry
            'OSOAA.View.Phi': 90.0,      # Relative azimuth angle (degrees)
            'OSOAA.View.Level': -1,       # Output level (-1 = just below surface)
            'OSOAA.View.Z': 0.0,          # Depth for output (0 = at surface)
            
            # Angular discretization
            'ANG.Thetas': solar_zenith,   # Solar zenith angle
            'ANG.Rad.NbGauss': 40,        # Number of Gauss angles for radiance
            'ANG.Mie.NbGauss': 80,        # Number of Gauss angles for Mie
            
            # Atmospheric profile
            'PROFILE.Atm.Pressure': 1013.25,  # Surface pressure (hPa)
            'PROFILE.Atm.HR': 8.0,            # Rayleigh scale height (km)
            'PROFILE.Atm.Ha': 2.0,            # Aerosol scale height (km)
            'PROFILE.Atm.MO3': 0.3,           # Ozone column (atm-cm)
            
            # Aerosol model (mono-modal log-normal distribution)
            'AER.Model': 0,
            'AER.MMD.MRwa': 1.45,         # Real refractive index
            'AER.MMD.MIwa': 0.001,        # Imaginary refractive index
            'AER.MMD.SDtype': 2,          # Log-normal distribution
            'AER.MMD.SDradius': 0.15,     # Modal radius (µm)
            'AER.MMD.SDvar': 0.6,         # Standard deviation
            'AER.AOTref': aot,            # AOT at reference wavelength
            'AER.Waref': 550.0,           # Reference wavelength (nm)
            'AER.Angexp': 1.0,            # Angstrom exponent
            
            # Sea profile
            'PROFILE.Sea.Depth': 100.0,   # Ocean depth (m)
            
            # Hydrosol model (phytoplankton)
            'HYD.Model': 0,               # Model type
            'HYD.Chl': chlorophyll,       # Chlorophyll concentration (mg/m³)
            'HYD.ProfilChl': 0,           # Homogeneous profile
            
            # Sea surface
            'SEA.Ind': 1.334,             # Refractive index of seawater
            'SEA.Type': 1,                # Rough surface (Cox & Munk)
            'SEA.Wind': wind_speed,       # Wind speed (m/s)
            
            # Output files
            'SOS.ResFile.Bin': 'LUM_SF.bin',
            'OSOAA.ResFile.vsVZA': 'LUM_vsVZA.txt',
            'OSOAA.ResFile.vsZ': 'LUM_vsZ.txt',
            'OSOAA.ResFile.Advanced': 'LUM_Advanced.txt',
        }
    
    def create_input_file(self, params: Dict) -> str:
        """Generate OSOAA input file content from parameter dictionary."""
        lines = ['# OSOAA Simulation - Generated by Python', '']
        for key, value in params.items():
            lines.append(f"{key} {value}")
        return '\n'.join(lines)
    
    def run(self, params: Dict, verbose: bool = True) -> Dict:
        """
        Run OSOAA simulation with given parameters.
        
        Returns dictionary with parsed results.
        """
        # Create input file
        input_content = self.create_input_file(params)
        input_file = self.work_dir / 'osoaa_input.txt'
        
        with open(input_file, 'w') as f:
            f.write(input_content)
        
        if verbose:
            print(f"Running OSOAA simulation...")
            print(f"  Wavelength: {params.get('OSOAA.Wa', 'N/A')} nm")
            print(f"  Chlorophyll: {params.get('HYD.Chl', 'N/A')} mg/m³")
            print(f"  Solar zenith: {params.get('ANG.Thetas', 'N/A')}°")
        
        # Run simulation
        with open(input_file, 'r') as f_in:
            result = subprocess.run(
                [str(self.exe_path)],
                stdin=f_in,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                cwd=str(self.work_dir),
                env={**os.environ, 'OSOAA_ROOT': str(self.osoaa_root)}
            )
        
        if result.returncode != 0:
            print(f"OSOAA Error (return code {result.returncode}):")
            print(result.stderr[:2000] if result.stderr else "No stderr")
            raise RuntimeError(f"OSOAA simulation failed")
        
        if verbose:
            print("  Simulation completed successfully!")
        
        # Parse results
        return self.parse_results(params)
    
    def parse_results(self, params: Dict) -> Dict:
        """Parse OSOAA output files into numpy arrays."""
        results = {'params': params}
        
        # Parse LUM_vsVZA.txt (radiance vs viewing zenith angle)
        vza_file = self.work_dir / params.get('OSOAA.ResFile.vsVZA', 'LUM_vsVZA.txt')
        if vza_file.exists():
            results['vza_data'] = self._parse_vza_file(vza_file)
        
        # Parse LUM_vsZ.txt (radiance vs depth)
        vsz_file = self.work_dir / params.get('OSOAA.ResFile.vsZ', 'LUM_vsZ.txt')
        if vsz_file.exists():
            results['vsz_data'] = self._parse_vsz_file(vsz_file)
        
        return results
    
    def _parse_vza_file(self, filepath: Path) -> Dict:
        """
        Parse LUM_vsVZA.txt output file.
        
        Columns:
        1: Viewing zenith angle (degrees)
        2: I - Total intensity (Stokes I) - THIS IS THE RADIANCE
        3: Q - Stokes Q parameter
        4: U - Stokes U parameter
        5: Degree of Linear Polarization (DoLP)
        """
        # Count header lines (lines starting with #)
        with open(filepath, 'r') as f:
            lines = f.readlines()
        
        skip_rows = sum(1 for line in lines if line.strip().startswith('#') or not line.strip())
        
        try:
            data = np.loadtxt(filepath, skiprows=skip_rows)
        except:
            # Try to find data start
            data = np.loadtxt(filepath, comments='#')
        
        return {
            'vza': data[:, 0],          # Viewing Zenith Angle (degrees)
            'I': data[:, 1],            # Radiance (Stokes I) - W/m²/sr/nm or normalized
            'Q': data[:, 2],            # Stokes Q
            'U': data[:, 3],            # Stokes U
            'DoLP': data[:, 4] if data.shape[1] > 4 else None,  # Degree of polarization
        }
    
    def _parse_vsz_file(self, filepath: Path) -> Dict:
        """Parse LUM_vsZ.txt output file (radiance vs altitude/depth)."""
        try:
            data = np.loadtxt(filepath, comments='#')
            return {
                'z': data[:, 0],    # Altitude/depth
                'I': data[:, 1],    # Radiance
            }
        except:
            return None

print("OSOAASimulation class defined.")

## 3. Your First OSOAA Simulation: Clear Water

Let's run a simple simulation for clear ocean water (low chlorophyll) at 550 nm.

In [ ]:
# Initialize the simulation
sim = OSOAASimulation(OSOAA_ROOT)

# Get default parameters for clear water
params = sim.get_default_params(
    wavelength=550.0,      # Green light (nm)
    solar_zenith=30.0,     # Sun at 30° from zenith
    chlorophyll=0.05,      # Clear oligotrophic water (mg/m³)
    aot=0.05,              # Low aerosol loading
    wind_speed=3.0         # Calm sea surface (m/s)
)

print("Simulation parameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

In [ ]:
# Run the simulation
results = sim.run(params)

## 4. Visualizing Water-Leaving Radiance

The key output for ocean color applications is the **water-leaving radiance** - the radiance that emerges from the ocean after scattering by water molecules and particles (phytoplankton, suspended sediments, etc.).

In [ ]:
def plot_radiance_angular(results: Dict, title: str = None):
    """
    Plot water-leaving radiance as a function of viewing zenith angle.
    """
    if 'vza_data' not in results:
        print("No VZA data available")
        return
    
    vza = results['vza_data']['vza']
    radiance = results['vza_data']['I']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(vza, radiance, 'b-', linewidth=2, marker='o', markersize=4)
    ax.set_xlabel('Viewing Zenith Angle (degrees)', fontsize=12)
    ax.set_ylabel('Radiance (Stokes I)', fontsize=12)
    
    if title:
        ax.set_title(title, fontsize=14)
    else:
        wl = results['params'].get('OSOAA.Wa', '?')
        chl = results['params'].get('HYD.Chl', '?')
        ax.set_title(f'Water-Leaving Radiance at {wl} nm\nChl = {chl} mg/m³', fontsize=14)
    
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 90)
    
    # Add nadir value annotation
    nadir_idx = np.argmin(np.abs(vza))
    nadir_rad = radiance[nadir_idx]
    ax.axhline(y=nadir_rad, color='r', linestyle='--', alpha=0.5, label=f'Nadir: {nadir_rad:.4f}')
    ax.legend()
    
    plt.tight_layout()
    return fig, ax

# Plot the results
plot_radiance_angular(results)
plt.show()

## 5. Effect of Chlorophyll Concentration

Chlorophyll concentration strongly affects water-leaving radiance, especially in the blue and green wavelengths. Let's explore this effect.

In [ ]:
# Chlorophyll concentration series: from ultra-oligotrophic to eutrophic
chl_values = [0.03, 0.1, 0.3, 1.0, 3.0, 10.0]  # mg/m³

chl_results = {}

print("Running chlorophyll series simulations...\n")
for chl in chl_values:
    params = sim.get_default_params(
        wavelength=550.0,
        solar_zenith=30.0,
        chlorophyll=chl,
        aot=0.05,
        wind_speed=3.0
    )
    # Update output filename to avoid overwriting
    params['OSOAA.ResFile.vsVZA'] = f'LUM_vsVZA_chl{chl}.txt'
    
    try:
        chl_results[chl] = sim.run(params, verbose=False)
        print(f"  Chl = {chl:5.2f} mg/m³ - Done")
    except Exception as e:
        print(f"  Chl = {chl:5.2f} mg/m³ - Failed: {e}")

print("\nAll simulations completed!")

In [ ]:
def plot_chlorophyll_series(chl_results: Dict):
    """
    Compare water-leaving radiance for different chlorophyll concentrations.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Color map for chlorophyll values
    colors = plt.cm.viridis(np.linspace(0, 1, len(chl_results)))
    
    nadir_radiances = []
    chl_vals = []
    
    # Plot 1: Angular distribution
    for (chl, result), color in zip(sorted(chl_results.items()), colors):
        if 'vza_data' in result:
            vza = result['vza_data']['vza']
            rad = result['vza_data']['I']
            axes[0].plot(vza, rad, '-', color=color, linewidth=2, 
                        label=f'Chl = {chl} mg/m³')
            
            # Get nadir value
            nadir_idx = np.argmin(np.abs(vza))
            nadir_radiances.append(rad[nadir_idx])
            chl_vals.append(chl)
    
    axes[0].set_xlabel('Viewing Zenith Angle (degrees)', fontsize=12)
    axes[0].set_ylabel('Radiance (Stokes I)', fontsize=12)
    axes[0].set_title('Angular Distribution of Water-Leaving Radiance\n(550 nm)', fontsize=12)
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(0, 80)
    
    # Plot 2: Nadir radiance vs Chlorophyll
    if nadir_radiances:
        axes[1].semilogx(chl_vals, nadir_radiances, 'o-', color='darkgreen', 
                        linewidth=2, markersize=10)
        axes[1].set_xlabel('Chlorophyll Concentration (mg/m³)', fontsize=12)
        axes[1].set_ylabel('Nadir Radiance (Stokes I)', fontsize=12)
        axes[1].set_title('Nadir Water-Leaving Radiance vs Chlorophyll\n(550 nm)', fontsize=12)
        axes[1].grid(True, alpha=0.3)
        
        # Add water type annotations
        axes[1].axvspan(0.01, 0.1, alpha=0.2, color='blue', label='Oligotrophic')
        axes[1].axvspan(0.1, 1.0, alpha=0.2, color='green', label='Mesotrophic')
        axes[1].axvspan(1.0, 10.0, alpha=0.2, color='brown', label='Eutrophic')
        axes[1].legend(loc='upper left')
    
    plt.tight_layout()
    return fig

if chl_results:
    plot_chlorophyll_series(chl_results)
    plt.show()

## 6. Spectral Water-Leaving Radiance

Ocean color remote sensing relies on the spectral shape of water-leaving radiance. Let's simulate across the visible spectrum.

In [ ]:
# Wavelength range (visible spectrum)
wavelengths = [412, 443, 490, 510, 555, 620, 670, 750]  # Common ocean color bands (nm)

spectral_results = {}

print("Running spectral simulations...\n")
for wl in wavelengths:
    params = sim.get_default_params(
        wavelength=float(wl),
        solar_zenith=30.0,
        chlorophyll=0.3,  # Moderate chlorophyll
        aot=0.05,
        wind_speed=3.0
    )
    params['AER.Waref'] = float(wl)  # Match reference wavelength
    params['OSOAA.ResFile.vsVZA'] = f'LUM_vsVZA_wl{wl}.txt'
    
    try:
        spectral_results[wl] = sim.run(params, verbose=False)
        print(f"  λ = {wl} nm - Done")
    except Exception as e:
        print(f"  λ = {wl} nm - Failed: {e}")

print("\nSpectral simulations completed!")

In [ ]:
def plot_spectrum(spectral_results: Dict, viewing_angle: float = 0.0):
    """
    Plot water-leaving radiance spectrum at a given viewing angle.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    wavelengths = []
    radiances = []
    
    for wl, result in sorted(spectral_results.items()):
        if 'vza_data' in result:
            vza = result['vza_data']['vza']
            rad = result['vza_data']['I']
            
            # Find radiance at specified viewing angle
            idx = np.argmin(np.abs(vza - viewing_angle))
            wavelengths.append(wl)
            radiances.append(rad[idx])
    
    if wavelengths:
        # Create color gradient matching wavelength
        ax.plot(wavelengths, radiances, 'k-', linewidth=2, zorder=1)
        
        # Add colored points
        for wl, rad in zip(wavelengths, radiances):
            color = wavelength_to_rgb(wl)
            ax.scatter([wl], [rad], c=[color], s=100, zorder=2, edgecolors='black')
        
        ax.set_xlabel('Wavelength (nm)', fontsize=12)
        ax.set_ylabel('Water-Leaving Radiance (Stokes I)', fontsize=12)
        ax.set_title(f'Spectral Water-Leaving Radiance\n(VZA = {viewing_angle}°, Chl = 0.3 mg/m³)', fontsize=14)
        ax.grid(True, alpha=0.3)
        ax.set_xlim(400, 800)
        
    plt.tight_layout()
    return fig

def wavelength_to_rgb(wavelength: float) -> tuple:
    """
    Convert wavelength (nm) to approximate RGB color.
    """
    if wavelength < 380:
        return (0.5, 0, 0.5)
    elif wavelength < 440:
        return (0.5 - 0.5*(wavelength-380)/(440-380), 0, 1)
    elif wavelength < 490:
        return (0, (wavelength-440)/(490-440), 1)
    elif wavelength < 510:
        return (0, 1, 1-(wavelength-490)/(510-490))
    elif wavelength < 580:
        return ((wavelength-510)/(580-510), 1, 0)
    elif wavelength < 645:
        return (1, 1-(wavelength-580)/(645-580), 0)
    elif wavelength < 780:
        return (1, 0, 0)
    else:
        return (0.5, 0, 0)

if spectral_results:
    plot_spectrum(spectral_results, viewing_angle=0.0)
    plt.show()

## 7. Polarization of Water-Leaving Radiance

OSOAA is a **vector** radiative transfer code, meaning it computes all four Stokes parameters (I, Q, U, V). The degree of linear polarization (DoLP) provides additional information about the scattering regime.

In [ ]:
def plot_polarization(results: Dict):
    """
    Plot Stokes parameters and degree of linear polarization.
    """
    if 'vza_data' not in results:
        print("No VZA data available")
        return
    
    data = results['vza_data']
    vza = data['vza']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Stokes I (total intensity)
    axes[0, 0].plot(vza, data['I'], 'b-', linewidth=2)
    axes[0, 0].set_xlabel('VZA (degrees)')
    axes[0, 0].set_ylabel('Stokes I')
    axes[0, 0].set_title('Stokes I (Total Intensity)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Stokes Q
    axes[0, 1].plot(vza, data['Q'], 'r-', linewidth=2)
    axes[0, 1].set_xlabel('VZA (degrees)')
    axes[0, 1].set_ylabel('Stokes Q')
    axes[0, 1].set_title('Stokes Q (Linear Polarization)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Stokes U
    axes[1, 0].plot(vza, data['U'], 'g-', linewidth=2)
    axes[1, 0].set_xlabel('VZA (degrees)')
    axes[1, 0].set_ylabel('Stokes U')
    axes[1, 0].set_title('Stokes U (Linear Polarization)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Degree of Linear Polarization
    if data['DoLP'] is not None:
        axes[1, 1].plot(vza, data['DoLP'] * 100, 'm-', linewidth=2)
        axes[1, 1].set_ylabel('DoLP (%)')
    else:
        # Calculate DoLP from Q and U
        dolp = np.sqrt(data['Q']**2 + data['U']**2) / data['I'] * 100
        axes[1, 1].plot(vza, dolp, 'm-', linewidth=2)
        axes[1, 1].set_ylabel('DoLP (%)')
    
    axes[1, 1].set_xlabel('VZA (degrees)')
    axes[1, 1].set_title('Degree of Linear Polarization')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

if results and 'vza_data' in results:
    plot_polarization(results)
    plt.show()

## 8. Key Physical Insights

### Water-Leaving Radiance (Lw)

The **water-leaving radiance** is the radiance that emerges from the ocean surface after:
1. Solar radiation entering the water
2. Multiple scattering by water molecules and particles
3. Absorption by water, phytoplankton (chlorophyll), dissolved organic matter (CDOM), and suspended particles
4. Transmission through the air-sea interface

### Factors Affecting Lw:

| Factor | Effect on Lw |
|--------|-------------|
| **Chlorophyll** | Increases absorption in blue (443 nm), enhances green reflectance |
| **CDOM** | Strong absorption in blue, decreases Lw at short wavelengths |
| **Sediments** | Increases backscattering, raises Lw especially in red |
| **Solar angle** | Higher SZA = longer path = more atmospheric effects |
| **Viewing angle** | Off-nadir viewing changes Fresnel reflectance |
| **Wind speed** | Affects surface roughness and sun glint patterns |

## 9. Remote Sensing Reflectance

For ocean color remote sensing, we often work with **remote sensing reflectance** (Rrs), which normalizes the water-leaving radiance by the downwelling irradiance:

$$R_{rs} = \frac{L_w}{E_d}$$

where $E_d$ is the downwelling irradiance just above the surface.

In [ ]:
# Note: To compute Rrs, you would need to also compute or specify Ed.
# OSOAA can output the downwelling irradiance through appropriate parameter settings.

def estimate_rrs_from_osoaa(results: Dict, Ed: float = None):
    """
    Estimate remote sensing reflectance from OSOAA output.
    
    If Ed is not provided, this returns the normalized radiance which
    is proportional to Rrs.
    """
    if 'vza_data' not in results:
        return None
    
    vza = results['vza_data']['vza']
    Lw = results['vza_data']['I']
    
    # Get nadir value
    nadir_idx = np.argmin(np.abs(vza))
    Lw_nadir = Lw[nadir_idx]
    
    if Ed is not None:
        Rrs = Lw_nadir / Ed
        print(f"Nadir water-leaving radiance: {Lw_nadir:.6f}")
        print(f"Remote sensing reflectance (Rrs): {Rrs:.6f} sr⁻¹")
        return Rrs
    else:
        print(f"Nadir water-leaving radiance (normalized): {Lw_nadir:.6f}")
        return Lw_nadir

if results:
    estimate_rrs_from_osoaa(results)

## 10. Summary and Next Steps

### What we learned:

1. **OSOAA** computes full Stokes vector radiance for coupled atmosphere-ocean systems
2. **Water-leaving radiance** is the key output for ocean color applications
3. **Chlorophyll** strongly affects the spectral shape of Lw
4. **Angular distribution** of Lw depends on scattering phase functions
5. **Polarization** provides additional diagnostic information

### Suggested exercises:

1. Compare different aerosol types (maritime vs. continental)
2. Explore the effect of wind speed on surface reflectance
3. Simulate different solar zenith angles (morning vs. noon)
4. Add CDOM or suspended sediments to the simulation
5. Compare results with MODIS or VIIRS ocean color products

### Resources:

- OSOAA documentation: See `docs/` folder in the OSOAA installation
- Reference: Chami et al. (2015), Opt. Express 23, 27829-27852
- Parameter reference: `docs/user_guide/input_parameters.rst`

In [ ]:
# Clean up temporary files (optional)
import shutil

cleanup = False  # Set to True to remove temporary files

if cleanup and hasattr(sim, 'work_dir'):
    print(f"Cleaning up: {sim.work_dir}")
    shutil.rmtree(sim.work_dir, ignore_errors=True)
else:
    print(f"Temporary files preserved in: {sim.work_dir}")